In [6]:
from ensmallen_graph.datasets.linqs import Cora, CiteSeer, PubMedDiabetes
from experiment.utils import load_graph_and_features
from embiggen.utils import compute_node_embedding
from embiggen.node_prediction import NoLaN
from ensmallen_graph.utils import holdouts_generator
from plot_keras_history import plot_history
from tqdm.auto import trange

In [7]:
graph, features = load_graph_and_features(PubMedDiabetes)

Extracting words features:   0%|          | 0/20217 [00:00<?, ?it/s]

In [11]:
node_embedding_method_name = "CBOW"

In [12]:
node_embedding, _ = compute_node_embedding(
    graph,
    node_embedding_method_name=node_embedding_method_name,
    verbose=2,
    automatically_drop_unsupported_parameters=True,
    explore_weight=2,
    return_weight=0.5,
    negative_samples=100
)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

In [ ]:
histories = []
all_performance = []
for holdout_number, (train, validation) in holdouts_generator(
    graph.node_label_holdout,
    holdouts_number=10
):
    nolan = NoLaN(
        graph,
        node_embedding=node_embedding,
        node_features=features,
        node_embedding_dropout_rate=0.2,
        node_features_dropout_rate=0.2,
        l1_kernel_regularization=1e-4,
        hidden_dense_layers=256
    )
    #nolan.summary()
    histories.append(nolan.fit(
        train,
        validation_graph=validation,
        #verbose=False
    ))
    all_performance.append(
        nolan.evaluate(train, validation_graph=validation)
    )
all_performance = pd.concat(all_performance)

Computing holdouts:   0%|          | 0/10 [00:00<?, ?it/s]

0epoch [00:00, ?epoch/s]

In [ ]:
all_performance[all_performance.run_type=="validation"].accuracy.hist()

In [ ]:
plot_history(histories)